In [1]:
%run ./Preprocessing.ipynb

import optuna
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import (
    StratifiedKFold, TimeSeriesSplit, KFold
)
from sklearn.model_selection import cross_val_score


Preprocessed : xtrain, xtest, ytrain, ytest
(74736, 13) (18684, 13) (74736,) (18684,)


c:\dev\.conda\envs\jupyter-lab\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [63]:
train_x

array([[ 0.33377727,  0.57453868,  0.        ,  1.        ,  1.        ,
         0.        ],
       [ 2.39675074,  0.17263624,  0.        ,  1.        ,  1.        ,
         0.        ],
       [ 0.52718104,  0.4490531 ,  0.        ,  1.        ,  1.        ,
         0.        ],
       ...,
       [-1.14898491,  0.26454281,  0.        ,  1.        ,  0.        ,
         1.        ],
       [ 1.88100737,  0.81638891,  1.        ,  0.        ,  1.        ,
         0.        ],
       [ 0.46271311,  0.6513986 ,  0.        ,  1.        ,  1.        ,
         0.        ]])

In [64]:
train_y

0         1
1         0
2         1
3         0
4         0
         ..
381104    0
381105    0
381106    0
381107    0
381108    0
Name: Response, Length: 381109, dtype: int64

In [65]:
test_x

array([[ 0.33377727,  0.57453868,  0.        ,  1.        ,  1.        ,
         0.        ],
       [ 2.39675074,  0.17263624,  0.        ,  1.        ,  1.        ,
         0.        ],
       [ 0.52718104,  0.4490531 ,  0.        ,  1.        ,  1.        ,
         0.        ],
       ...,
       [-1.14898491,  0.26454281,  0.        ,  1.        ,  0.        ,
         1.        ],
       [ 1.88100737,  0.81638891,  1.        ,  0.        ,  1.        ,
         0.        ],
       [ 0.46271311,  0.6513986 ,  0.        ,  1.        ,  1.        ,
         0.        ]])

In [66]:
test_y

0         1
1         0
2         1
3         0
4         0
         ..
381104    0
381105    0
381106    0
381107    0
381108    0
Name: Response, Length: 381109, dtype: int64

In [38]:
optgrid = {
            'cv': KFold(n_splits=10, random_state=None, shuffle=True),
            'number_of_trials': 2,
            'time_constraint': 60 * 1
        }

In [47]:
 def opt_search_sklearn(estimator, X: pd.DataFrame, y: pd.Series, opt_grid = optgrid) -> optuna.Study:
        """
        Sklearn cv optuna method
        X: dataframe
        y: labels
        model_param_config: parameters search space and configuration
        
        
        """
        from sklearn.model_selection import (
            StratifiedKFold, TimeSeriesSplit, KFold
        )

        static_param = {
            "penalty": 'elasticnet',
            'dual': False,
            'verbose': -1,
        }

        def objective(trial):
            

            param_grid = {
                "C": trial.suggest_float('C', 1, 3),
                # "intercept_scaling": trial.suggest_float("intercept_scaling", 1, 1.0),
                "max_iter": trial.suggest_float("max_iter", 300, 300),
                "l1_ratio": trial.suggest_float("l1_ratio", 0.001, 0.9999)
            }
            
            full_grid = {**param_grid, **static_param}

            cv = cross_val_score(estimator=estimator,
                                 X=X,
                                 y=y,
                                 cv= opt_grid['cv'],
                                 scoring='roc_auc',
                                 verbose=True)

            score = cv.mean()

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

            return score

        study = optuna.create_study(sampler=optuna.samplers.TPESampler(), direction='maximize')
        study.optimize(objective,
                       n_trials=opt_grid['number_of_trials'],
                       show_progress_bar=False,
                       timeout=opt_grid['time_constraint'])

        return study

In [48]:
opt_search_sklearn(LogisticRegression(),train_x,train_y)

[I 2022-07-30 16:54:19,447] A new study created in memory with name: no-name-f488f9db-6a60-4a97-991f-1ece136f4c12
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
c:\dev\.conda\envs\jupyter-lab\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,
c:\dev\.conda\envs\jupyter-lab\lib\site-packages\sklearn\linear_model\_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
 

In [67]:
lr = LogisticRegression()

In [68]:
lr.fit(train_x, train_y)

LogisticRegression()

In [69]:
pred = lr.predict(test_x)

In [70]:
pred

array([0, 0, 0, ..., 0, 0, 0], dtype=int64)

In [71]:
lr.score(test_y,pred)

ValueError: Expected 2D array, got 1D array instead:
array=[1 0 1 ... 0 0 0].
Reshape your data either using array.reshape(-1, 1) if your data has a single feature or array.reshape(1, -1) if it contains a single sample.

In [72]:
from sklearn.metrics import precision_score,recall_score,accuracy_score,f1_score,confusion_matrix,roc_auc_score,classification_report

print(classification_report(test_y,pred))

              precision    recall  f1-score   support

           0       0.88      1.00      0.93    334399
           1       0.24      0.00      0.00     46710

    accuracy                           0.88    381109
   macro avg       0.56      0.50      0.47    381109
weighted avg       0.80      0.88      0.82    381109

